In [37]:
from service.sector_rotation import *
import os

In [38]:
def extract_tickmatch(start_date: str = None, end_date: str = None) -> pd.DataFrame:
    query = """
        SELECT 
            st.Symbol,
            st.Type,
            st.Last,
            st.Vol,
            st.Time
        FROM stocksm_tickmatchs st
    """
    conditions = []
    params = ()

    if start_date:
        conditions.append("`Time` >= %s")
        params += (f"{start_date} 00:00:00",)
    
    if end_date:
        conditions.append("`Time` <= %s")
        params += (f"{end_date} 23:59:59",)
    
    if conditions:
        query += " WHERE " + " AND ".join(conditions)

    try:
        result = pd.read_sql(query, conn, params=params)
        return result
    except Exception as ex:
        print("Connection could not be made due to the following error: \n", ex)

In [39]:
def extract_symbols() -> pd.DataFrame:
    query = """
        select 
            ss.SymbolName ,
            ss.SectorName 
        from sm_stocks ss 
    """
    try:
        result = pd.read_sql(query, conn)
        return result
    except Exception as ex:
        print("Connection could not be made due to the following error: \n", ex)

In [40]:
if os.path.isfile('test_data.csv'):
    tickmatch_df = pd.read_csv('test_data.csv')[['Symbol','Type','Last','Vol','Time']]
else :
    tickmatch_df = extract_tickmatch('2023.07.22','2023.07.29')
    tickmatch_df.to_csv('test_data.csv')

tickmatch_df.head()

,Symbol,Type,Last,Vol,Time
0,1INCHTHB,SELL,11.52,311,2023.07.22 00:04.000
1,1INCHTHB,SELL,11.52,836,2023.07.22 00:06.000
2,1INCHTHB,BUY,11.62,311,2023.07.22 00:07.000
3,1INCHTHB,BUY,11.62,100,2023.07.22 00:07.000
4,1INCHTHB,SELL,11.62,894,2023.07.22 00:07.000


In [41]:
if os.path.isfile('symbols.csv'):
    symbol_df = pd.read_csv('symbols.csv')[['SymbolName','SectorName']]
else :
    symbol_df = extract_symbols()
    symbol_df.to_csv('symbols.csv')
    
symbol_df.head()

,SymbolName,SectorName
0,24CS,NaN
1,2S,STEEL
2,3K-BAT,AUTO
3,7UP,ENERG
4,A,PROP


In [42]:
tickmatch_df['Vol'] = tickmatch_df.where(tickmatch_df['type'] == 'SELL', -tickmatch_df['Vol'])

KeyError: 'type'